In [1]:
# file related
import os
from os.path import join
import datetime

# machine learning
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import Audio as play_audio
import scipy as sp
import taunet_utils
import json

# RTNeural special import
import sys
sys.path.append("../RTNeural/python/")
from model_utils import save_model

print(f"TF version {tf.__version__}")

TF version 2.12.0


In [2]:
# input_data, output_data = taunet_utils.create_dataset(join("dataset", "audio"), 
#                                                       join("dataset", "human_input", "AITD_Dataset_Kristof_beta_1.csv"),
#                                                       join("..", "src", "af", "AFInC.dll"),
#                                                       join("dataset", "saved", "AITD_Dataset_Kristof_beta_1.json"))

with open(join("dataset", "saved", "AITD_Dataset_Kristof_beta_1in.json"), "r") as json_file:
    input_data = json.load(json_file)
with open(join("dataset", "saved", "AITD_Dataset_Kristof_beta_1out.json"), "r") as json_file:
    output_data = json.load(json_file)

for d in input_data:
    print(f"{d}: {input_data[d]} -> {output_data[d]}")

1: [123.9669, 0.012, 0.2251, 15648, 0.7536, 10, 1, 1, 1, 47.5291, 0.6738, 0.8758, 550.0, 350.0] -> [360.5123, 1144.7427]
10: [159.0106, 0.0131, 0.2123, 2440, 0.8535, 10, 1, 1, 1, 45.9809, 0.3157, 0.8187, 550.0, 350.0] -> [330.558, 1814.8958]
11: [117.4935, 0.0408, 0.31, 7351, 0.7748, 10, 1, 1, 1, 43.3603, 0.7365, 0.7567, 550.0, 350.0] -> [302.5441, 430.5534]
12: [172.4138, 0.3676, 0.4617, 15840, 1.569, 10, 1, 1, 1, 35.7993, 0.7287, 0.8574, 550.0, 350.0] -> [70.1614, 1192.9986]
13: [156.7944, 0.0098, 0.1759, 3139, 0.859, 10, 1, 1, 1, 43.1486, 0.109, 0.9997, 550.0, 350.0] -> [283.9686, 1571.1926]
14: [152.5424, 0.0422, 0.245, 2145, 1.1746, 10, 1, 1, 1, 20.3692, 0.6313, 0.2773, 550.0, 350.0] -> [281.9786, 461.7476]
15: [143.77, 1.4763, 0.1339, 2145, 2.2429, 10, 1, 1, 1, 4.7266, 0.5662, 0.8151, 550.0, 350.0] -> [206.762, 1754.8867]
16: [127.8409, 0.0328, 0.2358, 2134, 0.8128, 10, 1, 1, 1, 36.023, 0.6062, 0.2058, 550.0, 350.0] -> [499.6292, 320.9579]
17: [140.1869, 0.0419, 0.1801, 9275, 1.0

In [3]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    # model.add(layers.LSTM(64, return_sequences=True))   # short term memory, useful if input data is related accross vectors
    model.add(Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.001))) # no activation (linear): continuous mapping of outputs (this is not a classification task!)
    return model

input_shape = (14,)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                960       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
 dense_6 (Dense)             (None, 2)                 1

In [4]:
combined_data = [(input_data[key], output_data[key]) for key in input_data.keys()]

train_data, test_data = train_test_split(combined_data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.05, random_state=42)

log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

class LogRMSECallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(LogRMSECallback, self).__init__()
        self.log_dir = log_dir

    def on_epoch_end(self, epoch, logs=None):
        mse = logs.get('val_loss')
        rmse = np.sqrt(mse)
        with tf.summary.create_file_writer(self.log_dir).as_default():
            tf.summary.scalar('val_rmse', rmse, step=epoch)

%load_ext tensorboard
%tensorboard --logdir logs/ --port 6006

history = model.fit(
    x=np.array([item[0] for item in train_data]),
    y=np.array([item[1] for item in train_data]),
    validation_data=(
        np.array([item[0] for item in val_data]),
        np.array([item[1] for item in val_data])
    ),
    epochs=80,
    # callbacks=[tensorboard_callback, early_stopping_callback],
    callbacks=[tensorboard_callback, early_stopping_callback, LogRMSECallback(join(log_dir, "rmse"))],
    # batch_size=16
)

Reusing TensorBoard on port 6006 (pid 11700), started 1 day, 23:41:06 ago. (Use '!kill 11700' to kill it.)

Epoch 1/80
2/2 [==============================] - 2s 472ms/step - loss: 1819069.6250 - accuracy: 0.1200 - val_loss: 1010760.3125 - val_accuracy: 0.3333
Epoch 2/80
2/2 [==============================] - 0s 164ms/step - loss: 1376281.0000 - accuracy: 0.1600 - val_loss: 791135.6875 - val_accuracy: 0.6667
Epoch 3/80
2/2 [==============================] - 0s 239ms/step - loss: 1225723.8750 - accuracy: 0.8800 - val_loss: 755092.1875 - val_accuracy: 0.6667
Epoch 4/80
2/2 [==============================] - 0s 167ms/step - loss: 1181065.6250 - accuracy: 0.8800 - val_loss: 700328.2500 - val_accuracy: 0.6667
Epoch 5/80
2/2 [==============================] - 0s 175ms/step - loss: 1117524.0000 - accuracy: 0.8800 - val_loss: 680392.6250 - val_accuracy: 0.6667
Epoch 6/80
2/2 [==============================] - 0s 164ms/step - loss: 1097269.2500 - accuracy: 0.8800 - val_loss: 665930.4375 - val_accuracy: 0.6667
Epoch 7/80
2/2 [==============================] - 0s 157ms/step - loss: 1078688.6250 - accura

In [5]:
train_metrics = model.evaluate(
    np.array([item[0] for item in train_data]),
    np.array([item[1] for item in train_data]),
    verbose=0
)
val_metrics = model.evaluate(
    np.array([item[0] for item in val_data]),
    np.array([item[1] for item in val_data]),
    verbose=0
)
test_metrics = model.evaluate(
    np.array([item[0] for item in test_data]),
    np.array([item[1] for item in test_data]),
    verbose=0
)

for item in test_data:
    inp = np.array([item[0]])  # Reshape input data into a batch
    prediction = model.predict(inp)
    print("Input:", item[0])
    print("Expected Output:", item[1])
    print("Predicted Output:", prediction)


print({f"Train {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, train_metrics)})
print({f"Validation {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, val_metrics)})
print({f"Test {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, test_metrics)})
print("RMSE for test set:", np.sqrt(test_metrics[0]))

# save model as .h5, .tflite and .json
model.save(join(log_dir, "taunet.pb"))
tflite_model = tf.lite.TFLiteConverter.from_saved_model(join(log_dir, "taunet.pb")).convert()
with open(join(log_dir, "taunet.tflite"), 'wb') as f:
  f.write(tflite_model)
save_model(model, join(log_dir, "taunet.json"))

1/1 [==============================] - 0s 128ms/step
Input: [123.9669, 0.012, 0.2251, 15648, 0.7536, 10, 1, 1, 1, 47.5291, 0.6738, 0.8758, 550.0, 350.0]
Expected Output: [360.5123, 1144.7427]
Predicted Output: [[0.14437272 0.15437375]]
1/1 [==============================] - 0s 30ms/step
Input: [152.5424, 0.0422, 0.245, 2145, 1.1746, 10, 1, 1, 1, 20.3692, 0.6313, 0.2773, 550.0, 350.0]
Expected Output: [281.9786, 461.7476]
Predicted Output: [[-1.2076904 -0.1405368]]
1/1 [==============================] - 0s 23ms/step
Input: [114.7959, 0.0193, 0.1472, 10727, 0.5565, 10, 1, 1, 1, 173.1196, 0.1961, 0.9731, 550.0, 350.0]
Expected Output: [52.7942, 812.402]
Predicted Output: [[0.14437272 0.15437375]]
1/1 [==============================] - 0s 30ms/step
Input: [150.0, 0.015, 0.1608, 12284, 0.8636, 10, 1, 1, 1, 67.5176, 0.5384, 0.8216, 550.0, 350.0]
Expected Output: [429.106, 423.9346]
Predicted Output: [[0.14437272 0.15437375]]
1/1 [==============================] - 0s 25ms/step
Input: [167.286

INFO:tensorflow:Assets written to: logs/20240427-152953\taunet.pb\assets


INFO:tensorflow:Assets written to: logs/20240427-152953\taunet.pb\assets


: 